<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW3_%E5%BE%85%E8%BE%A6%E6%B8%85%E5%96%AE%E8%88%87%E7%95%AA%E8%8C%84%E9%90%98%E7%B4%80%E9%8C%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **待辦清單與番茄鐘紀錄（作業三）**
目標：在 Sheet 維護 To-do（狀態、預估時間、完成時間）→ Colab 輸出今日計畫與完成率 → 寫回完成標記。

AI 點子：讓模型把今日任務排成三段行動計畫（morning/afternoon/evening）。


In [1]:
# 環境設立
!pip install --upgrade -q google-generativeai
!pip install -q gspread oauth2client pandas plotly gradio

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import time
import datetime
import json
import plotly.express as px
import gradio as gr
from google.colab import userdata
import google.generativeai as genai
from IPython.display import Markdown, display
import sys


In [8]:
# Google Sheet 連線與操作
def connect_to_sheet():
    try:
        scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']

        creds = ServiceAccountCredentials.from_json_keyfile_name('gen-lang-client-0156076368-e711a064f70d.json', scope)
        client = gspread.authorize(creds)

        sheet = client.open("待辦清單與番茄鐘紀錄").sheet1
        print("已連線")
        return sheet, None
    except Exception as e:
        error_msg = f"連線失敗"
        print(error_msg)
        return None, error_msg

def get_tasks_with_calculated_status(sheet):
    if sheet is None: return pd.DataFrame()
    try:
        records = sheet.get_all_records()
        now = datetime.datetime.now()
        for task in records:
            if task['狀態'] not in ['完成', '進行中']:
                try:
                    start_time = datetime.datetime.strptime(task['任務開始時間'], '%Y-%m-%d %H:%M')
                    task['狀態'] = '休息' if now < start_time else '待辦'
                except (ValueError, TypeError): task['狀態'] = '待辦'
        return pd.DataFrame(records)
    except Exception as e: print(f"讀取任務失敗：{e}"); return pd.DataFrame()

def reconstruct_records_from_sheet(tasks_df):
    """掃描任務列表，將已完成的任務轉換成番茄鐘紀錄格式"""
    if tasks_df.empty:
        return []

    completed_tasks = tasks_df[tasks_df['狀態'] == '完成']
    reconstructed_records = []
    for index, task in completed_tasks.iterrows():
        # 進行基本的資料驗證
        if task.get('完成時間') and task.get('預估時間 (分鐘)'):
            record = {
                "task_name": task['任務名稱'],
                "start_time": "N/A (from Sheet)", # 歷史資料無此紀錄
                "end_time": task['完成時間'],
                "duration_minutes": task['預估時間 (分鐘)'] # 使用預估時間作為近似值
            }
            reconstructed_records.append(record)
    return reconstructed_records

def add_task(sheet, name, st, et): return f"新增：'{name}'" if sheet and sheet.append_row([name, st, '待辦', int(et), '']) else "Sheet連線失敗"
def update_task_status(sheet, name, status, time=None):
    if not sheet: return "Sheet連線失敗"
    try:
        cell = sheet.find(name); sheet.update_cell(cell.row, 3, status)
        if status == '完成' and time: sheet.update_cell(cell.row, 5, time)
    except Exception as e: print(f"更新 '{name}' 狀態失敗：{e}")
def delete_task(sheet, name): return f"刪除：'{name}'" if sheet and sheet.delete_rows(sheet.find(name).row) else "Sheet連線失敗"


In [5]:
# 檔案
def save_to_file(records, selected_tasks, file_format='csv'):
    if not records: return "沒有任何已完成的紀錄可匯出。"
    records_to_export = [r for r in records if not selected_tasks or r['task_name'] in selected_tasks]
    if not records_to_export: return "沒有符合條件的紀錄可匯出。"
    filename = f"pomodoro_records.{file_format}"; df = pd.DataFrame(records_to_export)
    if file_format == 'csv': df.to_csv(filename, index=False, encoding='utf-8-sig')
    else: df.to_json(filename, orient='records', indent=4, force_ascii=False)
    return f"成功匯出 {len(records_to_export)} 筆紀錄至 {filename}！"

def load_from_file(app_state, upload_file):
    if upload_file is None: return "未上傳檔案。", app_state
    try:
        if upload_file.name.endswith('.csv'): imported = pd.read_csv(upload_file.name).to_dict('records')
        else:
            with open(upload_file.name, 'r', encoding='utf-8') as f: imported = json.load(f)
        all_records = app_state['pomodoro_records'] + imported
        app_state['pomodoro_records'] = pd.DataFrame(all_records).drop_duplicates().to_dict('records')
        return f"成功匯入 {len(imported)} 筆紀錄！", app_state
    except Exception as e: return f"匯入失敗：{e}", app_state

# 視覺化
def visualize_stats(app_state):
    records = app_state['pomodoro_records']
    if not records: return None, "沒有紀錄可供視覺化。"
    df = pd.DataFrame(records); summary = df.groupby('task_name')['duration_minutes'].sum().reset_index()
    fig = px.bar(summary, x='task_name', y='duration_minutes', title='各任務總花費時間統計 (分鐘)')
    return fig, "圖表已生成"

# AI 函式
def configure_ai():
    try:
        # 【修改點 3/3：請確認 Colab 密鑰中 'GEMINI_API_KEY' 已設定】
        api_key = userdata.get('GEMINI_API_KEY')
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-pro-latest')
        print("AI 模型已設定")
        return model, None
    except Exception as e:
        error_msg = f"AI 模型設定失敗"; print(error_msg); return None, error_msg

def generate_ai_plan_wrapper(sheet, model, ai_error_msg):
    if model is None: return ai_error_msg if ai_error_msg else "AI 模型未設定。"
    df = get_tasks_with_calculated_status(sheet); pending_tasks = df[df['狀態'].isin(['待辦', '休息'])].to_dict('records')
    if not pending_tasks: return "太棒了！今天沒有待辦任務。"
    task_list_str = "\n".join([f"- {task['任務名稱']} (預估 {task['預估時間 (分鐘)']} 分鐘)" for task in pending_tasks])
    prompt = f"你是一位專業的時間管理教練...(內容省略)...今日待辦任務清單：\n{task_list_str}"
    try:
        response = model.generate_content(prompt); return response.text
    except Exception as e: return f"AI 生成計畫失敗"


In [6]:
# 3. 初始化應用狀態
sheet_instance, sheet_error = connect_to_sheet()
gemini_model, ai_error_message = configure_ai()
INITIAL_APP_STATE = {"timer_running": False, "current_task_name": None, "timer_start_time": None, "pomodoro_duration_seconds": 25 * 60, "pomodoro_records": []}

已連線
AI 模型已設定


In [10]:
# Gradio 應用程式定義與綁定

# 只有在成功連線到 Sheet 後才啟動 Gradio
if sheet_instance:
    with gr.Blocks(theme=gr.themes.Soft(), title="待辦事項與番茄鐘") as demo:
        # --- UI 元件與狀態定義 ---
        app_state = gr.State(value=INITIAL_APP_STATE)
        gr.Markdown("# 📝 待辦事項與番茄鐘 Pro"); status_textbox = gr.Textbox(label="系統狀態", interactive=False, value="應用程式已啟動。")
        with gr.Tabs():
            with gr.TabItem("🕒 總覽與計時"):
                tasks_df_display = gr.DataFrame(interactive=False, label="任務列表"); refresh_button = gr.Button("🔄 從 Google Sheet 刷新")
                with gr.Row(): timer_task_dropdown = gr.Dropdown(choices=[], label="選擇要開始的任務"); timer_display = gr.Textbox("25:00", label="計時器", interactive=False)
                with gr.Row(): start_button = gr.Button("▶️ 開始番茄鐘", variant="primary"); stop_button = gr.Button("⏹️ 停止/中斷", variant="stop", interactive=False)
            with gr.TabItem("➕➖ 新增/刪除任務"):
                with gr.Row():
                    with gr.Column(): gr.Markdown("### 新增任務"); add_name_input = gr.Textbox(label="任務名稱"); add_start_time_input = gr.Textbox(label="任務開始時間", placeholder="格式：YYYY-MM-DD HH:MM"); add_est_time_input = gr.Number(label="預估時間 (分鐘)", value=25); add_button = gr.Button("新增", variant="primary")
                    with gr.Column(): gr.Markdown("### 刪除任務"); delete_task_dropdown = gr.Dropdown(choices=[], label="選擇要刪除的任務"); delete_button = gr.Button("刪除", variant="stop")
            with gr.TabItem("💾 匯出/匯入紀錄"):
                gr.Markdown("### 匯出已完成的番茄鐘紀錄"); export_task_selector = gr.Dropdown(choices=[], multiselect=True, label="選擇要匯出的任務 (可多選，不選則匯出全部)")
                with gr.Row(): export_select_all_button = gr.Button("全選", variant="secondary"); export_csv_button = gr.Button("匯出為 CSV"); export_json_button = gr.Button("匯出為 JSON")
                gr.Markdown("---"); gr.Markdown("### 從檔案匯入紀錄"); import_file_input = gr.File(label="上傳 CSV/JSON 檔案"); import_button = gr.Button("開始匯入")
            with gr.TabItem("📊 統計視覺化"):
                visualize_button = gr.Button("生成統計圖表"); plot_display = gr.Plot(label="任務時間分佈圖")
            with gr.TabItem("🤖 AI 行動計畫"):
                generate_ai_plan_button = gr.Button("🚀 幫我規劃今天的行動！", variant="primary"); ai_plan_display = gr.Markdown(value=ai_error_message if ai_error_message else "點擊按鈕。")

        # --- Gradio Wrapper and Event Listeners ---
        def start_timer_wrapper(current_app_state, task_name):
            if not task_name: return current_app_state, "⚠️ 請選擇任務", gr.update(), gr.update(), gr.update(), gr.update()
            if current_app_state["timer_running"]: return current_app_state, "⚠️ 任務進行中", gr.update(), gr.update(), gr.update(), gr.update()
            current_app_state["timer_running"] = True; current_app_state["current_task_name"] = task_name; current_app_state["timer_start_time"] = time.time()
            update_task_status(sheet_instance, task_name, "進行中")
            minutes, seconds = divmod(current_app_state['pomodoro_duration_seconds'], 60)
            return current_app_state, f"✅ 開始：{task_name}", f"{minutes:02d}:{seconds:02d}", gr.update(interactive=False), gr.update(interactive=True), get_tasks_with_calculated_status(sheet_instance)

        def stop_timer_wrapper(current_app_state, is_manual_stop=True):
            if not current_app_state["timer_running"]: return current_app_state, "⚠️ 計時器未啟動", gr.update(), gr.update(), gr.update(), gr.update(), gr.update()
            task_name = current_app_state["current_task_name"]; duration = time.time() - current_app_state["timer_start_time"]
            record = {"task_name": task_name, "start_time": datetime.datetime.fromtimestamp(current_app_state["timer_start_time"]).strftime('%Y-%m-%d %H:%M:%S'), "end_time": datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "duration_minutes": round(duration / 60, 2)}
            current_app_state['pomodoro_records'].append(record)
            if not is_manual_stop:
                update_task_status(sheet_instance, task_name, "完成", datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')); status_msg = f"🎉 完成：{task_name}"
            else:
                update_task_status(sheet_instance, task_name, "待辦"); status_msg = f"⏸️ 停止：{task_name}"
            current_app_state["timer_running"] = False; current_app_state["current_task_name"] = None
            export_choices = sorted(list(set([r['task_name'] for r in current_app_state['pomodoro_records']])))
            return current_app_state, status_msg, "25:00", gr.update(interactive=True), gr.update(interactive=False), get_tasks_with_calculated_status(sheet_instance), gr.update(choices=export_choices)

        def timer_loop_generator(current_app_state):
            while current_app_state["timer_running"]:
                elapsed = time.time() - current_app_state["timer_start_time"]; remaining_seconds = max(0, current_app_state["pomodoro_duration_seconds"] - int(elapsed))
                if remaining_seconds == 0:
                    current_app_state, status_msg, time_str, start_btn, stop_btn, tasks_df, export_choices = stop_timer_wrapper(current_app_state, is_manual_stop=False)
                    yield {app_state: current_app_state, status_textbox: status_msg, timer_display: time_str, start_button: start_btn, stop_button: stop_btn, tasks_df_display: tasks_df, export_task_selector: export_choices}; break
                else:
                    minutes, seconds = divmod(remaining_seconds, 60); yield {app_state: current_app_state, timer_display: f"{minutes:02d}:{seconds:02d}"}; time.sleep(1)
            yield {app_state: current_app_state}

        def refresh_all_ui(current_app_state):
            """刷新所有UI元件，並從Sheet重建歷史紀錄"""
            df = get_tasks_with_calculated_status(sheet_instance)
            main_choices = df['任務名稱'].tolist() if not df.empty else []
            reconstructed_records = reconstruct_records_from_sheet(df)
            all_records = reconstructed_records + current_app_state['pomodoro_records']
            current_app_state['pomodoro_records'] = pd.DataFrame(all_records).drop_duplicates(subset=['task_name', 'end_time']).to_dict('records')
            export_choices = sorted(list(set([r['task_name'] for r in current_app_state['pomodoro_records']])))
            return current_app_state, df, gr.update(choices=main_choices), gr.update(choices=main_choices), gr.update(choices=export_choices)

        def refresh_export_ui(current_app_state):
            export_choices = sorted(list(set([r['task_name'] for r in current_app_state['pomodoro_records']])))
            return gr.update(choices=export_choices)

        def select_all_export_tasks(current_app_state):
            choices = sorted(list(set([r['task_name'] for r in current_app_state['pomodoro_records']])))
            return gr.update(value=choices)

        # --- Event Listeners ---
        demo.load(fn=refresh_all_ui, inputs=[app_state], outputs=[app_state, tasks_df_display, timer_task_dropdown, delete_task_dropdown, export_task_selector])

        start_button.click(fn=start_timer_wrapper, inputs=[app_state, timer_task_dropdown], outputs=[app_state, status_textbox, timer_display, start_button, stop_button, tasks_df_display]).then(fn=timer_loop_generator, inputs=[app_state], outputs={app_state, status_textbox, timer_display, start_button, stop_button, tasks_df_display, export_task_selector})
        stop_button.click(fn=stop_timer_wrapper, inputs=[app_state], outputs=[app_state, status_textbox, timer_display, start_button, stop_button, tasks_df_display, export_task_selector])
        refresh_button.click(fn=refresh_all_ui, inputs=[app_state], outputs=[app_state, tasks_df_display, timer_task_dropdown, delete_task_dropdown, export_task_selector])

        add_button.click(lambda n, s, e, state: (add_task(sheet_instance, n, s, e), *refresh_all_ui(state)), inputs=[add_name_input, add_start_time_input, add_est_time_input, app_state], outputs=[status_textbox, app_state, tasks_df_display, timer_task_dropdown, delete_task_dropdown, export_task_selector])
        delete_button.click(lambda n, state: (delete_task(sheet_instance, n), *refresh_all_ui(state)), inputs=[delete_task_dropdown, app_state], outputs=[status_textbox, app_state, tasks_df_display, timer_task_dropdown, delete_task_dropdown, export_task_selector])

        export_csv_button.click(fn=save_to_file, inputs=[app_state, export_task_selector, gr.State('csv')], outputs=[status_textbox])
        export_json_button.click(fn=save_to_file, inputs=[app_state, export_task_selector, gr.State('json')], outputs=[status_textbox])
        export_select_all_button.click(fn=select_all_export_tasks, inputs=[app_state], outputs=[export_task_selector])

        import_button.click(fn=load_from_file, inputs=[app_state, import_file_input], outputs=[status_textbox, app_state]).then(fn=refresh_all_ui, inputs=[app_state], outputs=[tasks_df_display, timer_task_dropdown, delete_task_dropdown, export_task_selector])
        visualize_button.click(fn=visualize_stats, inputs=[app_state], outputs=[plot_display, status_textbox])
        generate_ai_plan_button.click(fn=lambda: generate_ai_plan_wrapper(sheet_instance, gemini_model, ai_error_message), outputs=[ai_plan_display])

    # 啟動
    print("\n✅ 正在啟動 Gradio 應用程式...")
    demo.launch(share=True, debug=True)
else:
    print("\n❌ Google Sheet 連線失敗，無法啟動 Gradio 應用程式。")


✅ 正在啟動 Gradio 應用程式...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0e9297c3cb0c843a74.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0e9297c3cb0c843a74.gradio.live
